In [1]:
# Dependencies and libraries #

import os
import requests
import json
import pandas as pd
import datetime
from datetime import date, timedelta
import hvplot
import alpaca_trade_api as tradeapi
from finta import TA
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

%matplotlib inline

In [2]:
# Set Alpaca API key and secret by calling the os.getenv function and referencing the environment variable names
# Set each environment variable to a notebook variable of the same name
alpaca_api_key = "PKT126TVACUEXXFT8Z85"
alpaca_secret_key = "WwlEmn0WUNYY0iJL7ltRQypkzqNfojvcnFtScBIy"

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [3]:
# Create your Alpaca API REST object by calling Alpaca's tradeapi.REST` function
# Set the parameters to your alpaca_api_key, alpaca_secret_key and api_version="v2" 
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")


In [4]:
ticker = "TSLA"

In [5]:
# Dates
start_date = pd.Timestamp("2019-10-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-10", tz="America/New_York").isoformat()

In [6]:
# Timeframe 
timeframe = "1Day"

In [8]:
# Price information #
df = alpaca.get_bars(
    ticker,
    timeframe,
    start = start_date,
    end = end_date,
).df

# Preview the resulting `portfolio_prices_df` DataFrame. 
df.head()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-10-01 04:00:00+00:00,241.50,245.95,239.13,244.69,6209990,101685,242.666097
2019-10-02 04:00:00+00:00,243.29,244.65,239.43,243.13,6256548,94931,241.362570
2019-10-03 04:00:00+00:00,232.04,234.48,224.28,233.03,15137764,230597,229.266463
2019-10-04 04:00:00+00:00,231.82,234.78,228.07,231.43,8025180,106994,230.830586
2019-10-07 04:00:00+00:00,230.00,238.56,228.55,237.72,8086958,97332,234.987850


In [9]:
#create OHLCV df; drop 'trade_count' and 'vwap'columns
df = df.drop(['trade_count', 'vwap'], axis=1)

# Use the pct_change function to generate the returns from "close"
df["actual_returns"] = df["close"].pct_change()

# Drop all NaN values from the DataFrame
df = df.dropna()

# Generate the Input Features, X
# Create additional technical indicators
df['sma_slow'] = TA.SMA(df, 100)
df['sma_fast'] = TA.SMA(df, 4)
df["ssma"] = TA.SSMA(df)
df["ema"] = TA.EMA(df, 50)
df["dema"] = TA.DEMA(df)
df["tema"] = TA.TEMA(df)
df["trima"] = TA.TRIMA(df)
df["trix"] = TA.TRIX(df)
df["vama"] = TA.VAMA(df)
df["kama"] = TA.KAMA(df)
df["zlema"] = TA.ZLEMA(df)
df["wma"] = TA.WMA(df)

df = df.dropna()

df.head()

,open,high,low,close,volume,actual_returns,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,,,,,,,
2020-02-25 05:00:00+00:00,849.00,856.60,787.00,799.91,17320981,-0.040634,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556
2020-02-26 05:00:00+00:00,782.45,813.31,776.11,778.80,14163418,-0.026390,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333
2020-02-27 05:00:00+00:00,730.21,739.77,669.00,679.00,24277832,-0.128146,456.8824,772.8750,774.704328,644.116213,785.502081,741.621826,691.737315,1.830608,837.616079,818.665230,855.457409,807.005333
2020-02-28 05:00:00+00:00,625.81,690.52,611.52,667.99,24566968,-0.016215,461.2480,731.4250,762.847116,645.067891,742.013136,692.104305,704.958056,1.744067,813.707288,806.553958,833.670395,774.653111
2020-03-02 05:00:00+00:00,711.53,743.69,686.67,743.62,20201438,0.113220,466.3070,717.3525,760.710760,648.993922,738.446086,699.553803,716.517191,1.654336,783.969571,805.307392,825.239568,760.361111


In [10]:
# Assign a copy of the technical variable columns to a new DataFrame called `X` and shift values.
# The shifted 'X' values will align a prior day's X values with the next day's 'y'/returns/trading signal,
# to train the predictive model. 
X = df[['sma_slow', 'sma_fast', 'ssma', 'ema', 'dema', 'tema', 'trima', 'trix', 'vama', 'kama', 'zlema', 'wma']].shift().dropna().copy()

# Initialize a `signal` column
df['signal'] = 0.0

# signal values will be based on the daily returns: positive returns yield '1', negative returns '-1'
df.loc[(df['actual_returns'] >= 0), 'signal'] = 1
df.loc[(df['actual_returns'] < 0), 'signal'] = -1

# Copy the 'signal' column to a new Series called `y`.
y = df['signal']

display(df.head(2))
display(X.head(2))
display(X.tail(2))
display(y.head(2))

,open,high,low,close,volume,actual_returns,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma,signal
timestamp,,,,,,,,,,,,,,,,,,,
2020-02-25 05:00:00+00:00,849.00,856.60,787.00,799.91,17320981,-0.040634,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556,-1.0
2020-02-26 05:00:00+00:00,782.45,813.31,776.11,778.80,14163418,-0.026390,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333,-1.0


,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,
2020-02-26 05:00:00+00:00,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556
2020-02-27 05:00:00+00:00,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333


,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,
2022-10-07 04:00:00+00:00,621.1966,242.6950,286.383655,444.280359,237.820321,240.814765,316.680463,-2.415128,258.923003,260.795585,227.340505,252.767556
2022-10-10 04:00:00+00:00,616.1836,237.8625,279.348804,435.605443,229.183932,230.356701,306.816358,-2.422937,252.514878,254.479060,221.266393,245.137111


timestamp
2020-02-25 05:00:00+00:00   -1.0
2020-02-26 05:00:00+00:00   -1.0
Name: signal, dtype: float64

In [11]:
# create the 'begin' and 'end' variables to split the training and testing datasets in the next step. 
# the training window is one year from the start of the index. 
training_begin = X.index.min()
print(training_begin)

training_end = X.index.min() + DateOffset(years=1)
print(training_end)

2020-02-26 05:00:00+00:00
2021-02-26 05:00:00+00:00


In [12]:
# Slice the 'X' dataframe and 'y' Series into congruous training datasets.
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Slice the testing 'X' and 'y' datasets, starting from the end of the training data until the most recent index. 
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Use StandardScaler to scale the X_train and X_test data.
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
